In [57]:
import scipy as sp
import numpy as np
import sympy as sy
import scipy.integrate as spi
from sympy.abc import t,x,y, z, w
import scipy.optimize as spo
from sympy.utilities.lambdify import lambdify 
import itertools as it
import time
import iminuit as im

In [60]:
#construct homotopy
def H(t, G, F, gamma):
    return [(1 - t)*G[i] + gamma*t*F[i] for i in range(len(G))]

#for degree 3 polynomial, do we need to generalise for more than one variable??
def G(x_array):
    func_listG = [i**3 - 1 for i in x_array]
    return func_listG

#construct inital starting systems for different number of starting systems
def F3(x_array):
    return [x_array[0]**3 - 6*x_array[1] + np.pi*x_array[2]**2, x_array[1]**3 + x_array[2]**2 + 8*x_array[0], x_array[2]**3 \
            + x_array[0]*x_array[1]*x_array[2] + np.pi]

def F4(x_array):
    return [x_array[0]**3 - 6*x_array[1] + np.pi*x_array[2]**2 + np.pi*x_array[3]**2, x_array[1]**3 + x_array[2]**2 + 8*x_array[0], x_array[2]**3 \
            + x_array[0]*x_array[1]*x_array[2] + np.pi, x_array[3]**3 - np.pi]

def F2predictortest(x_array):
    return [x_array[0] - 10, x_array[1] - 10]

def F2(x_array):
    return [x_array[0]**3 - 6*x_array[1] + np.pi, x_array[1]**3 + 8*x_array[0] + 76]

def F2Easy(x_array):
    return [x_array[0]**2  + np.pi, x_array[1]**2 - 76]

def F1(x_array):
    return [x_array[0]**3 + np.pi + x_array[0]**2 - x_array[0]]

def gamma_generate():
    real = np.random.rand()
    im = np.sqrt(1- real**2)
    return real + im*1j

def G_roots_find(n):
    root_list = [1, np.exp(1j*2*np.pi/3), np.exp(1j*2*np.pi*2/3)]
    return [i for i in it.product(root_list, repeat = n)]

In [61]:
def Homotopy_Continuation(t, x_array, F, N, tolerance = 1e-10, tolerance_zero = 1e-10, ratio_tolerance = 1e-5,\
                          N_ratio_tolerance = 50, debug = False):
    
    time_start = time.time()
    
    #count the number of roots
    num = 0
    
    delta_t = 1/N
    n = len(x_array)
    gamma = gamma_generate()
    gamma= 0.6 + 0.8j
    G_roots = G_roots_find(n)
    
    #construct homotopy
    H_func = H(t, G(x_array), F(x_array), gamma)
    
    #first derivative of H
    H_diff_x = sy.Matrix([[H_func[i].diff(x_array[j]) for i in range(len(x_array))] for j in range(len(x_array))])

    determinant_H = H_diff_x.det()
    
    #derivative with respect to t
    H_diff_t = sy.Matrix([H_func[i].diff(t) for i in range(len(x_array))])
    
    #check determinant is not zero so can invert
    if abs(determinant_H) == 0:
        raise TypeError('The determinant of H is zero!')
        
    det_H = lambdify((t, x_array), determinant_H)
    
    H_diff_x_inv = H_diff_x**-1
    H_diff_inv = -H_diff_x_inv*H_diff_t
    
    H_Hdiffprime = H_diff_x_inv*sy.Matrix(H_func)
    
    H_Hdiffprime = lambdify((t, x_array), [H_Hdiffprime[i] for i in range(len(H_Hdiffprime))])
        
    H_prime = lambdify((t, x_array), [H_diff_inv[i] for i in range(len(H_diff_inv))])
    
    Hprime_x = lambdify((x_array,t), [H_diff_x[i] for i in range(len(H_diff_x))])    
    H_func_1d = lambdify((x_array,t), H_func)
    

    x_old_arrays = []
    x_olds = []
    
    #run for all roots of starting system
    for x_old in G_roots:
        x_old_array = []
        num += 1
            
        t_n = 0
        
        #run for all steps starting at t=0 ending at t=1
        while t_n < 1:

            x_old_array.append(x_old)
            t_old = t_n
            t_n += delta_t
            
            if n == 1:
                sol = spi.solve_ivp(H_prime, (t_old, t_n), x_old)
                sol_x = sol.y[-1][-1]
                
                H_func_1 = lambdify((x,t), H_func_1d([x], t)[0])
                Hprime_x_1 = lambdify((x,t), Hprime_x([x], t)[0])

                #x_old = [spo.newton(H_func_1, sol_x, fprime = Hprime_x_1, args=(t_n, ))]
                
            else:
                if abs(det_H(t_n, x_old)) < tolerance_zero:
                    raise TypeError('The determinant of H is zero!')
                
                #perform RK4 method
                x_old_predictor=x_old
                sol = spi.solve_ivp(H_prime, (t_old, t_n), x_old)
                sol_x = sol.y[:,-1]
                    
            #newton's method
            x_old = sol_x
            ratio = np.full(n, 1)
            N_ratio = 0
            delta = np.full(n, ratio_tolerance)
            
            #tolerance criteria for step size in Newton's Method
            while max(ratio) > ratio_tolerance and N_ratio < N_ratio_tolerance:
                if abs(det_H(t_n, x_old)) < tolerance_zero:
                    raise TypeError('The determinant of H is zero!')
                x_old_intermediate = x_old - H_Hdiffprime(t_n, x_old)
                
                delta_old = delta
                delta = abs(x_old_intermediate - x_old)
                ratio = [delta[j]/(delta_old[j] + 1e-10) for j in range(n)]
                x_old = x_old_intermediate
                N_ratio += 1
                #x_old = spo.newton(H_func, sol_x, fprime = Hprime_x, args=(t_n, ))
            if debug:

                print('Newtons method took ', N_ratio, ' at step', t_old, " to ", t_n  )
                print('Predictor started at', x_old_predictor)
                print('predicted solution', sol_x)
                print('reached solution', x_old)
        #check root is found
        remainder = list(map(abs, F(x_old))) 

        #if all(rem < tolerance for rem in remainder) is True:
        if True:
            x_old = [x_old[i].real if abs(x_old[i].imag) < tolerance_zero else x_old[i] for i in range(len(x_old))]
            
            print('Root {}, \n{}, \nAccuracy {}!!\n'.format(num, x_old, remainder))
            x_olds.append(x_old)
            x_old_arrays.append(x_old_array)

    time_end = time.time()
    
    print('It took {} s to run!'.format(time_end - time_start))
    
    return x_olds, x_old_arrays

In [76]:
x_sols, x_sols_arrays = Homotopy_Continuation(t, [x,y,z, w], F4, N=50, tolerance = 1, N_ratio_tolerance= 200, ratio_tolerance= 1e-5,debug=False)

C:\Anaconda3\lib\site-packages\numpy\core\_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


Root 1, 
[(-2.4685902076594215+1.319151463571503j), (-0.9988599811289501+2.6405793870820577j), (0.05772905170646912-0.39007531262151063j), (-0.7322959437807613+1.268373780804881j)], 
Accuracy [7.549516567451064e-15, 5.0242958677880805e-15, 3.5596458096434965e-15, 3.1086244689504383e-15]!!

Root 2, 
[(-1.840410600053952-0.9558595365480638j), (-0.9234199035947459-2.3902618323466958j), (0.09440943013377769+0.5671302161919992j), (-0.7323295321687745+1.2688710884677967j)], 
Accuracy [0.004249891055544538, 0.004546371970927289, 0.0008452654454902675, 0.003208489703315079]!!

Root 3, 
[(-1.8406922308952505+0.956005978285031j), (-0.9231616425532543+2.390158265779125j), (0.09467139501525144-0.5670811045785284j), (-0.7327454926402265-1.2681794975784308j)], 
Accuracy [0.004173147942261003, 0.004463568644142535, 0.0008292229701054726, 0.0031516081416298174]!!

Root 4, 
[(-1.840410600053952-0.9558595365480638j), (-0.9234199035947459-2.3902618323466958j), (0.09440943013377769+0.5671302161919992j), (

Root 30, 
[(-2.4685902076594215+1.319151463571503j), (-0.9988599811289501+2.6405793870820577j), (0.05772905170646912-0.39007531262151063j), (-0.7322959437807613+1.268373780804881j)], 
Accuracy [7.549516567451064e-15, 5.0242958677880805e-15, 3.5596458096434965e-15, 3.1086244689504383e-15]!!

Root 31, 
[(2.6840906387113592+1.0508361629808596j), (1.1021645572807015+2.6181029602695127j), (-0.0028030761768309603+0.38388394409406396j), (-0.7322959437807612+1.268373780804881j)], 
Accuracy [7.601582815929793e-15, 1.3873790429220374e-14, 3.5605384226165826e-15, 3.66205343881779e-15]!!

Root 32, 
[(-2.4685902076594215+1.319151463571503j), (-0.9988599811289501+2.6405793870820577j), (0.05772905170646912-0.39007531262151063j), (-0.7322959437807613+1.268373780804881j)], 
Accuracy [7.549516567451064e-15, 5.0242958677880805e-15, 3.5596458096434965e-15, 3.1086244689504383e-15]!!

Root 33, 
[(-2.4685902076594215+1.319151463571503j), (-0.9988599811289501+2.6405793870820577j), (0.05772905170646912-0.39007

Root 59, 
[(-2.4685902076594224-1.319151463571502j), (-0.9988599811289508-2.6405793870820577j), (0.05772905170646902+0.39007531262151063j), (-0.7322959437807613-1.2683737808048805j)], 
Accuracy [1.8358671568380902e-14, 3.972054645195637e-15, 3.2986030318998278e-15, 5.924889052322839e-15]!!

Root 60, 
[(-2.4685902076594224-1.319151463571502j), (-0.9988599811289508-2.6405793870820577j), (0.05772905170646902+0.39007531262151063j), (-0.7322959437807613-1.2683737808048805j)], 
Accuracy [1.8358671568380902e-14, 3.972054645195637e-15, 3.2986030318998278e-15, 5.924889052322839e-15]!!

Root 61, 
[(-2.4685902076594215+1.319151463571503j), (-0.9988599811289501+2.6405793870820577j), (0.05772905170646912-0.39007531262151063j), (-0.7322959437807613+1.268373780804881j)], 
Accuracy [7.549516567451064e-15, 5.0242958677880805e-15, 3.5596458096434965e-15, 3.1086244689504383e-15]!!

Root 62, 
[(-2.4685902076594224-1.319151463571502j), (-0.9988599811289508-2.6405793870820577j), (0.05772905170646902+0.39007

In [66]:
coeff = [5,0,2,np.pi-6]
np.roots(coeff)

array([-0.33583382+0.85927473j, -0.33583382-0.85927473j,
        0.67166764+0.j        ])

In [7]:
coeff = [9,10,0,3]
np.roots(coeff)

array([-1.30641692+0.j        ,  0.0976529 +0.49559532j,
        0.0976529 -0.49559532j])

In [12]:
sp.__version__

'1.1.0'

In [73]:
def hhh(x,y):
    return (x - 3)**2 + np.pi + y**2

In [74]:
m = im.Minuit(hhh)
m.migrad()

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: InitialParamWarning: Parameter x does not have initial value. Assume 0.
  """Entry point for launching an IPython kernel.
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: InitialParamWarning: Parameter x is floating but does not have initial step size. Assume 1.
  """Entry point for launching an IPython kernel.
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: InitialParamWarning: Parameter y does not have initial value. Assume 0.
  """Entry point for launching an IPython kernel.
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: InitialParamWarning: Parameter y is floating but does not have initial step size. Assume 1.
  """Entry point for launching an IPython kernel.
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: InitialParamWarning: errordef is not given. Default to 1.
  """Entry point for launching an IPython kernel.


------------------------------------------------------------------
| FCN = 3.142                   |      Ncalls=24 (24 total)      |
| EDM = 9.46E-19 (Goal: 1E-05)  |            up = 1.0            |
------------------------------------------------------------------
|  Valid Min.   | Valid Param.  | Above EDM | Reached call limit |
------------------------------------------------------------------
|     True      |     True      |   False   |       False        |
------------------------------------------------------------------
| Hesse failed  |   Has cov.    | Accurate  | Pos. def. | Forced |
------------------------------------------------------------------
|     False     |     True      |   True    |   True    | False  |
------------------------------------------------------------------
------------------------------------------------------------------------------------------
|   | Name |   Value   | Hesse Err | Minos Err- | Minos Err+ | Limit-  | Limit+  | Fixed |
------------------------------------------------------------------------------------------
| 0 | x    |    3.0    |    1.0    |            |            |         |         |       |
| 1 | y    |    0.0    |    1.0    |            |            |         |         |       |
------------------------------------------------------------------------------------------